<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-и-подготовка-данных" data-toc-modified-id="Загрузка-и-подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка и подготовка данных</a></span><ul class="toc-item"><li><span><a href="#Импортируем-библиотеки-и-функции,-используемые-в-проекте" data-toc-modified-id="Импортируем-библиотеки-и-функции,-используемые-в-проекте-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Импортируем библиотеки и функции, используемые в проекте</a></span></li><li><span><a href="#Загрузка-и-предварительный-анализ-данных" data-toc-modified-id="Загрузка-и-предварительный-анализ-данных-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Загрузка и предварительный анализ данных</a></span></li><li><span><a href="#Подготовка-данных-без-использования-BERT" data-toc-modified-id="Подготовка-данных-без-использования-BERT-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Подготовка данных без использования BERT</a></span></li><li><span><a href="#Подготовка-данных-с-BERT" data-toc-modified-id="Подготовка-данных-с-BERT-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Подготовка данных с BERT</a></span></li></ul></li><li><span><a href="#Обучение-ML-моделей" data-toc-modified-id="Обучение-ML-моделей-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение ML-моделей</a></span><ul class="toc-item"><li><span><a href="#Обучение-ML-моделей-без-использования-BERT" data-toc-modified-id="Обучение-ML-моделей-без-использования-BERT-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Обучение ML-моделей без использования BERT</a></span></li><li><span><a href="#Обучение-ML-моделей-с-BERT" data-toc-modified-id="Обучение-ML-моделей-с-BERT-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Обучение ML-моделей с BERT</a></span></li></ul></li><li><span><a href="#Классификация-комментариев" data-toc-modified-id="Классификация-комментариев-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Классификация комментариев</a></span><ul class="toc-item"><li><span><a href="#Прогнозирование-ML-моделей-на-тестовых-выборках,-подготовленных-без-использования-BERT" data-toc-modified-id="Прогнозирование-ML-моделей-на-тестовых-выборках,-подготовленных-без-использования-BERT-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Прогнозирование ML-моделей на тестовых выборках, подготовленных без использования BERT</a></span></li><li><span><a href="#Прогнозирование-ML-моделей-на-тестовых-выборках,-подготовленных-с-BERT" data-toc-modified-id="Прогнозирование-ML-моделей-на-тестовых-выборках,-подготовленных-с-BERT-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Прогнозирование ML-моделей на тестовых выборках, подготовленных с BERT</a></span></li></ul></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Вывод</a></span></li></ul></div>

# Классификация комментариев для сервиса компании «BestService» с BERT

Компания «BestService» предоставляет сервис по поиску клиентов для специалистов в разных областях. Клиенты сервиса оставляют свои комментарии с предложениями по работе сервиса и качеству оказываемых услуг. Компании необходим инструмент, который будет выявлять негативные комментарии и отправлять их на модерацию.

Необходимо построить ML-модель для классификации комментариев на позитивные и негативные. В компании есть набор данных с разметкой комментариев.

Требования к ML-модели и данным:
1. Качество прогнозирования оценивать по метрике F1, значение метрики на тестовой выборке должно быть не меньше 0.75.
2. В рамках проекта подготовить обучающие и тестовые выборки для ML-моделей как с использованием предобученной модели нейронной сети BERT, так и без нее.

## Загрузка и подготовка данных

### Импортируем библиотеки и функции, используемые в проекте

In [1]:
import pandas as pd
import numpy as np
import math as mt
import re
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import transformers
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem import SnowballStemmer
from joblib import Parallel, delayed
from tqdm import notebook, tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import f1_score, make_scorer
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.dummy import DummyClassifier

### Загрузка и предварительный анализ данных

Прочитаем файл 'toxic_comments.csv' и сохраним его в переменной df_comments:

In [2]:
df_comments = pd.read_csv('toxic_comments.csv')

Выведем на экран общую информацию и первые 5 строк датафрейма df_comments:

In [3]:
df_comments.info()
df_comments.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


**По результатам предварительного анализа можно сделать следующие выводы:**
1. Датафрейм df_comments содержит 159571 строку и не содержит пропусков в данных. Каждая строка датафрейма содержит текст с комментарием и признак - положительный или отрицательный комментарий.
2. В комментариях используется английский язык.

### Подготовка данных без использования BERT

Произведем обработку текста комментариев (стемминг и очистку текста на английском языке) и сохраним результат в столбце 'stem_text' датафрейма df_comments:

In [4]:
#задаем размер батча
batch_size = 1000
#создаем текст разбитый на батчи
text_batch = [df_comments.loc[i: i + batch_size-1, 'text'] for i in range(0, df_comments.shape[0], batch_size)]
#создаем функцию для стемминга
eng_stemmer = SnowballStemmer('english')

def f_clear_text(f_text):
    #функция для очистки текста
    return ' '.join(re.sub(r'[^a-zA-Z| ]', ' ', f_text).split())

def f_eng_stem_clear(f_text_batch):
    #функция для стемминга текста на английском языке и его очистки от символов, кроме текста и одного пробела

    #объединяем в одну строку текст батча (батч содержит список из batch_size строк)
    merged_text = "||||".join(f_text_batch)
    #производим стемминг текста и его очистку (остаются английские слова с одним пробелом между ними и разделитель)
    stem_clear_text = f_clear_text(eng_stemmer.stem(merged_text))
    #разделяем одну строку на список строк (обратно после стемминга и очистки)
    list_txt = stem_clear_text.split('||||')

    return list_txt

In [5]:
#Создаем столбец stem_text в датафрейме df_comments после стемминга и очистки столбца text
list_text = Parallel(n_jobs=-1)(delayed(f_eng_stem_clear)(t) for t in tqdm(text_batch))
stem_text = sum(list_text, [])
df_comments['stem_text'] = stem_text
df_comments.info()

100%|████████████████████████████████████████████████████████████████████████████████| 160/160 [02:06<00:00,  1.26it/s]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   text       159571 non-null  object
 1   toxic      159571 non-null  int64 
 2   stem_text  159571 non-null  object
dtypes: int64(1), object(2)
memory usage: 3.7+ MB


Разделим датафрейм df_comments на обучающую и тестовую выборки и сохраним корпусы выборок:

In [6]:
#Сохранение датафрейма после подготовки stem_text во внешний файл при необходимости
#df_comments.to_csv('toxic_comments_stem.csv')
#Загрузка датафрейма с stem_text из внешнего файла при необходимости
#df_comments = pd.read_csv('toxic_comments_stem.csv')

#Разделяем датафрейм на обучающую и тестовую выборки и сохраняем корпусы выборок
df_train, df_test = train_test_split(df_comments, test_size=0.25)
corpus_train = df_train['stem_text'].values
corpus_test = df_test['stem_text'].values

Создадим признаки для ML-моделей: для этого посчитаем TF-IDF для корпусов и выделим целевой признак, также выведем на экран размеры обучающей и тестовой выборок: 

In [7]:
#Загрузим стоп-слова
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dominikana\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
#Посчитаем TF-IDF для корпусов (признаки модели)
tf_idf_vect = TfidfVectorizer(stop_words=stopwords).fit(corpus_train)
features_train = tf_idf_vect.transform(corpus_train)
features_test = tf_idf_vect.transform(corpus_test)

In [9]:
#Выделим целевой признак для обучающей и тестовой выборок
target_train = df_train['toxic']
target_test = df_test['toxic']

In [10]:
print(features_train.shape)
print(target_train.shape)
print(features_test.shape)
print(target_test.shape)

(119678, 142825)
(119678,)
(39893, 142825)
(39893,)


Посчитаем и выведем на экран баланс классов для обучающей и тестовой выборок:

In [11]:
print('Баланс классов для обучающей выборки:\n', target_train.value_counts(normalize=True))
print('Баланс классов для тестовой выборки:\n', target_test.value_counts(normalize=True))

Баланс классов для обучающей выборки:
 0    0.898427
1    0.101573
Name: toxic, dtype: float64
Баланс классов для тестовой выборки:
 0    0.898002
1    0.101998
Name: toxic, dtype: float64


**Вывод:**

1. Подготовлены без использования BERT обучающая и тестовая выборки для ML-моделей.
2. Присутствует дисбаланс классов выборок (0 - 90%, 1 - 10%; считается, что классы сбалансированы при приблизительно равном распределении).

### Подготовка данных с BERT

Для подготовки обучающей и тестовой выборок с использованием предобученной модели нейронной сети BERT необходимо: 
1. Произвести токенизацию каждого текста (текст разбивают на слова).
2. Затем токены передать модели BERT, которая переведёт их в векторные представления. Для этого модель обращается к составленному заранее словарю токенов. На выходе для каждого текста образуются векторы заданной длины (embeddings, «эмбеддинги»).
2. Затем выделить целевой признак и произвести разделение выборок на обучающую и тестовую.

Преобразование 1000 строк токенов с помощью BERT в эмбеддинги на персональном компьютере осуществляется в среднем 45 минут. Таким образом, создание эмбеддингов для всех строк датафрейма df_comments (159571 строка) с помощью BERT составит в среднем 7180 минут или около 5 суток.<br>
**Поэтому, в целях экономии времени на выполнение проекта выберем случайные 10000 строк из всей выборки, создадим для них эмбеддинги и сохраним их в отдельных файлах** ('features_bert10000.csv' - файл с эмбеддингами; 'comments_bert10000' - файл с целевым признаком).<br>
В целях демонстарции работы алгоритма создания эмбеддингов выберем случайные 100 строк и создадим для них эмбеддинги.<br>
Затем, для подготовки выборки на 10000 строк загрузим эмбеддинги и целевой признак из ранее сохраненных файлов.

In [12]:
#Выборка случайной части данных датафрейма для обработки моделью нейронной сети BERT
df_comments_bert = df_comments.sample(100).reset_index(drop=True)

#Загрузка предобученной модели нейронной сети BERT
config_bert = transformers.BertConfig.from_json_file('EngBERT/bert_config.json')
model_bert = transformers.BertModel.from_pretrained('EngBERT/pytorch_model.bin', config=config_bert)
tokenizer_bert = transformers.BertTokenizer(vocab_file='EngBERT/vocab.txt')

Some weights of the model checkpoint at EngBERT/pytorch_model.bin were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
#Создание токенов для текста
tokenized = df_comments_bert['text'].apply(
    lambda x: tokenizer_bert.encode(x, add_special_tokens=True, truncation=True, padding='max_length', max_length=512))

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

#Создание numpay array одинаковой длины для всех токенов и создание маски
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])
attention_mask = np.where(padded != 0, 1, 0)
print('padded.shape:', padded.shape, ', attention_mask:', attention_mask.shape)

padded.shape: (100, 512) , attention_mask: (100, 512)


In [14]:
#Создание embeddings с помощью модели нейронной сети BERT
batch_size = 50
embeddings = []

for i in notebook.tqdm(range(0, padded.shape[0], batch_size)):
    batch = torch.LongTensor(padded[i: i + batch_size])
    attention_mask_batch = torch.LongTensor(attention_mask[i: i + batch_size])
    
    with torch.no_grad():
        batch_embeddings = model_bert(batch, attention_mask=attention_mask_batch)
    
    embeddings.append(batch_embeddings[0][:,0,:].numpy())
#Создаем признаки
features_bert = np.concatenate(embeddings)

  0%|          | 0/2 [00:00<?, ?it/s]

In [15]:
#Загружаем ранее созданные выборки, в том числе с помощью BERT
df_features_bert = pd.read_csv('features_bert10000.csv')
df_comments_bert = pd.read_csv('comments_bert10000.csv')
features_bert = df_features_bert.values

#Разделяем признаки и целевой признак на обучающую и тестовую выборки
target_bert = df_comments_bert['toxic']
features_train_bert, features_test_bert, target_train_bert, target_test_bert = train_test_split(
    features_bert, target_bert, test_size=0.25)

In [16]:
print(features_train_bert.shape)
print(target_train_bert.shape)
print(features_test_bert.shape)
print(target_test_bert.shape)

(7500, 768)
(7500,)
(2500, 768)
(2500,)


Посчитаем и выведем на экран баланс классов для обучающей и тестовой выборок:

In [17]:
print('Баланс классов для обучающей выборки:\n', target_train_bert.value_counts(normalize=True))
print('Баланс классов для тестовой выборки:\n', target_test_bert.value_counts(normalize=True))

Баланс классов для обучающей выборки:
 0    0.892933
1    0.107067
Name: toxic, dtype: float64
Баланс классов для тестовой выборки:
 0    0.9016
1    0.0984
Name: toxic, dtype: float64


**Вывод:**

1. Подготовлены с использованием BERT обучающая и тестовая выборки для ML-моделей.
2. Присутствует дисбаланс классов выборок (0 - 90%, 1 - 10%; считается, что классы сбалансированы при приблизительно равном распределении).

## Обучение ML-моделей

Создадим функцию для обучения одной из 4-х моделей («логистическая регрессия», «градиентный бустинг» библиотек XGBoost, LightGBM и CatBoost), расчета метрики F1 и оценки качества модели кросс-валидацией:

In [18]:
def f_4ML_models_f1_cv(fv_features_train: pd.DataFrame, fv_target_train: pd.Series,
                       fv_modelname: str, fv_cv=5, fv_max_estimators=400, fv_learning_rate=0.3,
                       fv_scale_pos_weight=9, fv_class_weight='balanced'):
    #функция для обучения одной из 4-х моделей, расчета метрики F1 и оценки качества модели кросс-валидацией
    
    #определим одну из 4-х моделей
    if fv_modelname == 'XGBClassifier':
        model = XGBClassifier(random_state=12345, n_estimators=fv_max_estimators,
                              scale_pos_weight=fv_scale_pos_weight, learning_rate=fv_learning_rate,
                              use_label_encoder=False, verbosity=0)
    elif fv_modelname == 'LGBMClassifier':
        model = LGBMClassifier(random_state=12345, n_estimators=fv_max_estimators,
                               class_weight=fv_class_weight, learning_rate=fv_learning_rate)
    elif fv_modelname == 'CatBoostClassifier':
        model = CatBoostClassifier(random_state=12345, iterations=fv_max_estimators, learning_rate=fv_learning_rate,
                                   verbose=False)
    else:
        model = LogisticRegression(random_state=12345, solver='liblinear', class_weight=fv_class_weight)
    #вычислим метрику F1 с использованием кросс-валидации
    scorer = make_scorer(f1_score, greater_is_better=True)
    cv_f1score = cross_val_score(model, fv_features_train, fv_target_train, scoring=scorer, cv=fv_cv).mean()
    
    return cv_f1score

### Обучение ML-моделей без использования BERT

In [19]:
%%time
print('F1 модели LinearRegression =', f_4ML_models_f1_cv(features_train, target_train, 'LogisticRegression', 5))

F1 модели LinearRegression = 0.7494839910504182
Wall time: 5.43 s


In [20]:
%%time
print('F1 модели XGBClassifier =', f_4ML_models_f1_cv(features_train, target_train, 'XGBClassifier', 2, 400, 0.3, 9))

F1 модели XGBClassifier = 0.7542755763285248
Wall time: 5min 22s


In [21]:
%%time
print('F1 модели LGBMClassifier =', f_4ML_models_f1_cv(features_train, target_train, 'LGBMClassifier', 2, 400, 0.3))

F1 модели LGBMClassifier = 0.7515774930090837
Wall time: 2min 16s


In [22]:
%%time
print('F1 модели CatBoostClassifier =', f_4ML_models_f1_cv(features_train, target_train, 'CatBoostClassifier', 2, 400, 0.3))

F1 модели CatBoostClassifier = 0.7388022924781674
Wall time: 15min 19s


### Обучение ML-моделей с BERT

In [23]:
%%time
print('F1 модели LinearRegression =', f_4ML_models_f1_cv(features_train_bert, target_train_bert, 'LogisticRegression', 5))

F1 модели LinearRegression = 0.6401254987481935
Wall time: 14.6 s


In [24]:
%%time
print('F1 модели CatBoostClassifier =', f_4ML_models_f1_cv(features_train_bert, target_train_bert, 'CatBoostClassifier',
                                                           2, 312, 0.05))

F1 модели CatBoostClassifier = 0.5183258749666043
Wall time: 1min 46s


**Вывод:**

1. При обучении 4-х ML-моделей и оценке их качества кросс-валидацией с использованием выборок, подготовленных без использования BERT, достаточные для выполнения проекта значения метрики F1 показали 2 ML-модели - «градиентный бустинг» библиотек XGBoost и LightGBM (условие проекта: значение F1 не меньше 0.75).
2. При обучении 2-х ML-моделей и оценке их качества кросс-валидацией с использованием выборок, подготовленных с использованием предобученной модели нейронной сети BERT, обе ML-модели показали низкие значения метрики F1 (менее 0.75).

## Классификация комментариев

Создадим функцию для обучения одной из 4-х моделей («логистическая регрессия», «градиентный бустинг» библиотек XGBoost, LightGBM и CatBoost), расчета метрик F1 модели и констатной модели:

In [25]:
def f_5ML_models_f1(fv_features_train: pd.DataFrame, fv_target_train: pd.Series,
                    fv_features_test: pd.DataFrame, fv_target_test: pd.Series,
                    df_f1: pd.DataFrame, fv_bert:str, fv_modelname: str, fv_max_estimators=400, fv_learning_rate=0.3,
                    fv_scale_pos_weight=9, fv_class_weight='balanced'):
    #функция для обучения и расчета метрики F1 ML-моделей
    
    #определим одну из ML-моделей
    if fv_modelname == 'XGBClassifier':
        model = XGBClassifier(random_state=12345, n_estimators=fv_max_estimators,
                              scale_pos_weight=fv_scale_pos_weight, learning_rate=fv_learning_rate,
                              use_label_encoder=False, verbosity=0)
    elif fv_modelname == 'LGBMClassifier':
        model = LGBMClassifier(random_state=12345, n_estimators=fv_max_estimators,
                               class_weight=fv_class_weight, learning_rate=fv_learning_rate)
    elif fv_modelname == 'CatBoostClassifier':
        model = CatBoostClassifier(random_state=12345, iterations=fv_max_estimators, learning_rate=fv_learning_rate,
                                   custom_loss=['F1', 'AUC', 'Accuracy'])
    elif fv_modelname == 'LogisticRegression':
        model = LogisticRegression(random_state=12345, solver='liblinear', class_weight=fv_class_weight)
    else:
        #Константная модель
        model = DummyClassifier(strategy='uniform')
    #обучим модель
    if fv_modelname == 'CatBoostClassifier':
        model.fit(fv_features_train, fv_target_train, eval_set=(fv_features_test, fv_target_test),
                  verbose=False, plot=True)
    else:
        model.fit(fv_features_train, fv_target_train)
    #вычислим метрику F1
    f1score = f1_score(fv_target_test, model.predict(fv_features_test))
    #сохраним значения F1 для модели и константной модели
    df_f1.loc[df_f1['Модель']==fv_modelname + fv_bert, 'F1-score'] = f1score
    
    return f1score

Создадим датафрейм df_f1 для хранения значений метрик F1 моделей:

In [26]:
df_f1 = pd.DataFrame(data=[['LogisticRegression без BERT', np.nan],
                           ['XGBClassifier без BERT', np.nan],
                           ['LGBMClassifier без BERT', np.nan],
                           ['CatBoostClassifier без BERT', np.nan],
                           ['Константная модель без BERT', np.nan],
                           ['LogisticRegression с BERT', np.nan],
                           ['CatBoostClassifier с BERT', np.nan],
                           ['Константная модель с BERT', np.nan]],
                     columns=['Модель', 'F1-score'])

### Прогнозирование ML-моделей на тестовых выборках, подготовленных без использования BERT 

In [27]:
%%time
print('F1 модели LinearRegression =', f_5ML_models_f1(features_train, target_train, features_test, target_test,
                                                      df_f1, ' без BERT', 'LogisticRegression'))

F1 модели LinearRegression = 0.7542610571736785
Wall time: 1.63 s


In [28]:
%%time
print('F1 модели XGBClassifier =', f_5ML_models_f1(features_train, target_train, features_test, target_test,
                                                   df_f1, ' без BERT', 'XGBClassifier'))

F1 модели XGBClassifier = 0.7630605640314378
Wall time: 6min 9s


In [29]:
%%time
print('F1 модели LGBMClassifier =', f_5ML_models_f1(features_train, target_train, features_test, target_test,
                                                    df_f1, ' без BERT', 'LGBMClassifier'))

F1 модели LGBMClassifier = 0.7714790807307011
Wall time: 2min 13s


In [30]:
%%time
print('F1 модели CatBoostClassifier =', f_5ML_models_f1(features_train, target_train, features_test, target_test,
                                                        df_f1, ' без BERT', 'CatBoostClassifier'))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

F1 модели CatBoostClassifier = 0.7694467846110644
Wall time: 13min 54s


In [31]:
%%time
print('F1 константной модели =', f_5ML_models_f1(features_train, target_train, features_test, target_test,
                                                 df_f1, ' без BERT', 'Константная модель'))

F1 константной модели = 0.16886083309107522
Wall time: 52.4 ms


### Прогнозирование ML-моделей на тестовых выборках, подготовленных с BERT

In [32]:
%%time
print('F1 модели LinearRegression =', f_5ML_models_f1(features_train_bert, target_train_bert, features_test_bert,
                                                      target_test_bert, df_f1, ' с BERT', 'LogisticRegression'))

F1 модели LinearRegression = 0.6178861788617886
Wall time: 4.34 s


In [33]:
%%time
print('F1 модели CatBoostClassifier =', f_5ML_models_f1(features_train_bert, target_train_bert, features_test_bert,
                                                        target_test_bert, df_f1, ' с BERT', 'CatBoostClassifier',
                                                        312, 0.05))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

F1 модели CatBoostClassifier = 0.5659340659340659
Wall time: 1min 2s


In [34]:
%%time
print('F1 константной модели =', f_5ML_models_f1(features_train_bert, target_train_bert, features_test_bert,
                                                 target_test_bert, df_f1, ' с BERT', 'Константная модель'))

F1 константной модели = 0.16005291005291006
Wall time: 15.6 ms


## Вывод

Таблица значений метрик F1 моделей на тестовой выборке:

In [35]:
df_f1.head(10)

,Модель,F1-score
0,LogisticRegression без BERT,0.754261
1,XGBClassifier без BERT,0.763061
2,LGBMClassifier без BERT,0.771479
3,CatBoostClassifier без BERT,0.769447
4,Константная модель без BERT,0.168861
5,LogisticRegression с BERT,0.617886
6,CatBoostClassifier с BERT,0.565934
7,Константная модель с BERT,0.160053


**По результатам проекта можно сделать следующие выводы:**
1. Лучшей ML-моделью для классификации комментариев на положительные или отрицательные является «градиентный бустинг» библиотеки LightGBM, использующая выборки, подготовленные без BERT. Значение метрики F1 на тестовой выборке = 0.77, что соответствует условиям проекта (значение метрики F1 не меньше 0.75).
2. Если для компании приоритетом является скорость классификации комментариев, то целесообразно использовать модель «логистическая регрессия». У данной модели время обучения и прогнозирования - около 1.6 с (более чем в 80 раз быстрее, по сравнению с моделями градиентного бустинга), при этом качество предсказания по метрике F1 лишь немного хуже, чем у моделей градиентного бустинга.
3. Использование 10000 строк для подготовки выборок с использованием предобученной модели нейронной сети BERT, чтобы значение метрики F1 было не меньше 0.75, недостаточно. При этом,  следует отметить, что подготовка полной выборки с использованием BERT на персональном компьютере занимает около 5 суток.
4. Все ML-модели осуществляют прогноз лучше, чем константная модель (F1 = 0.17). Поэтому, применение указанных ML-моделей является оправданным для бизнеса.